In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
import pandas as pd
import numpy as np
import os
import re


In [2]:
qasm_path = r"./qasm_output"
qpy_path = r"./qpy_output"

In [3]:
def get_opt_level(filename:str):
    pat = re.compile(r'\w[0-9]')
    m = pat.search(filename)
    if m:
        num = m.group()[1:]
    else: 
        num = None
    return num

In [4]:
def get_num_qubits(filename:str):
    pat = re.compile(r'_\d+.')
    m = pat.search(filename)
    if m:
        num = m.group()[1:-1]
    else: 
        num = None
    return num

In [5]:
def parse_data(directory: str, filename:str):
    benchmark = filename.split("_")[0]
    num_qubits = get_num_qubits(filename)
    algorithm_flag =  "algorithm" in filename
    transpiled_flag = "transpiled" in filename 
    mapped_flag = "mapped" in filename
    opt_level = get_opt_level(filename)
    path = os.path.join(directory, filename)
    if ("ibm" in filename): gate_set="ibm"
    elif ("rigetti" in filename): gate_set="ibm"
    parsed_data = [benchmark, num_qubits, gate_set, algorithm_flag, transpiled_flag, mapped_flag, opt_level, path]
    return parsed_data

In [6]:
directory = qasm_path
rows_list = []
for filename in os.listdir(directory):
    if filename.endswith(".qasm") or filename.endswith(".qpy"):
        #print(filename)
        parsed_data = parse_data(directory, filename)
        rows_list.append(parsed_data)
        continue
    else:
        #print(filename)
        continue

In [7]:
colnames = ["benchmark", "num_qubits", "native_gate_set", "algo_layer", "transpiled_layer", "mapped_layer", "opt_level", "path"]

In [8]:
database = pd.DataFrame(rows_list, columns = colnames)
database['num_qubits'] = database['num_qubits'].astype(int)
database['opt_level'] = database['opt_level'].astype(int)
database

,benchmark,num_qubits,native_gate_set,algo_layer,transpiled_layer,mapped_layer,opt_level,path
0,ae,3,ibm,False,False,True,2,./qasm_output\ae_mapped_ibm_opt2_3.qasm
1,ae,3,ibm,False,True,False,2,./qasm_output\ae_transpiled_ibm_opt2_3.qasm
2,ae,4,ibm,False,True,False,2,./qasm_output\ae_transpiled_ibm_opt2_4.qasm
3,ae,5,ibm,False,True,False,2,./qasm_output\ae_transpiled_ibm_opt2_5.qasm
4,dj,3,ibm,False,False,True,2,./qasm_output\dj_mapped_ibm_opt2_3.qasm
5,dj,3,ibm,False,True,False,2,./qasm_output\dj_transpiled_ibm_opt2_3.qasm
6,dj,4,ibm,False,True,False,2,./qasm_output\dj_transpiled_ibm_opt2_4.qasm
7,dj,5,ibm,False,True,False,2,./qasm_output\dj_transpiled_ibm_opt2_5.qasm
8,ghz,3,ibm,False,False,True,2,./qasm_output\ghz_mapped_ibm_opt2_3.qasm
9,ghz,3,ibm,False,True,False,2,./qasm_output\ghz_transpiled_ibm_opt2_3.qasm


In [24]:
input_data ={'1': {'selectBench_1': 'Shor', 'qubitRange_1': '40', 'algorithmLevel_1': 'true', 'transpiledLevel_1': 'true', 'gate_set_1': 'rigetti', 'mappedLevel_1': 'true', 'optlevel_1"': '2'}, '3': {'selectBench_3': 'QFT', 'qubitRange_3': '62', 'transpiledLevel_3': 'true', 'gate_set_3': 'ibm', 'optlevel_3"': '2'}}

In [25]:
filter_list = []

for key, value in input_data.items():
    algorithm_flag = False
    transpiled_flag = False
    mapped_flag = False
    gate_set = False  
    name = list(value.values())[0]
    num_qubits = list(value.values())[1]
    for key, value in value.items():
        if "algorithmLevel" in key and value: algorithm_flag =True
        elif "transpiledLevel" in key and value: transpiled_flag =True 
        elif "mappedLevel" in key and value: mapped_flag =True
        
        if "gate_set" in key: gate_set = value 
            
        if "optlevel" in key: opt_level = value
    filter_list.append([name, num_qubits, gate_set, algorithm_flag, transpiled_flag, mapped_flag, opt_level])
    #print(name, qubit, gate_set, algorithm_flag, transpiled_flag, mapped_flag, opt_level )

In [26]:
colnames = ["benchmark", "num_qubits", "native_gate_set", "algo_layer", "transpiled_layer", "mapped_layer", "opt_level"]
filter_df = pd.DataFrame(filter_list, columns=colnames)
filter_df['num_qubits'] = filter_df['num_qubits'].astype(int)
filter_df['opt_level'] = filter_df['opt_level'].astype(int)
filter_df

,benchmark,num_qubits,native_gate_set,algo_layer,transpiled_layer,mapped_layer,opt_level
0,Shor,40,rigetti,True,True,True,2
1,QFT,62,ibm,False,True,False,2


In [58]:
def filterDatabase(filterDF, database):
    filter_df.values.tolist()
    filteredDB = pd.DataFrame(columns=colnames)
    for filter_params in filter_df.values.tolist():    
        tmp_res = database.loc[(database["benchmark"] == filter_params[0].lower()) 
                     & (database["num_qubits"] < filter_params[1])
                     & (database["native_gate_set"] == filter_params[2]) 
                     & (
                         ((database["algo_layer"] == filter_params[3]) 
                         & (database["transpiled_layer"] != filter_params[3]) 
                         & (database["mapped_layer"] != filter_params[3])) 
                     | 
                         ((database["algo_layer"] != filter_params[4]) 
                          & (database["transpiled_layer"] == filter_params[4]) 
                          & (database["mapped_layer"] != filter_params[4])) 
                     | 
                         ((database["algo_layer"] != filter_params[5]) 
                          & (database["transpiled_layer"] != filter_params[5]) 
                          & (database["mapped_layer"] ==  filter_params[5]))
                     )                
                     & (database["opt_level"] == filter_params[6]) 
                    ]
        filteredDB = pd.concat([filteredDB, tmp_res])
    print(filteredDB)
    return list(tmp_res.path)

In [60]:
file_paths = filterDatabase(filter_df, database)
file_paths

   benchmark num_qubits native_gate_set algo_layer transpiled_layer  \
25       qft          3             ibm      False             True   
26       qft          4             ibm      False             True   
27       qft          5             ibm      False             True   

   mapped_layer opt_level                                          path  
25        False         2  ./qasm_output\qft_transpiled_ibm_opt2_3.qasm  
26        False         2  ./qasm_output\qft_transpiled_ibm_opt2_4.qasm  
27        False         2  ./qasm_output\qft_transpiled_ibm_opt2_5.qasm  


['./qasm_output\\qft_transpiled_ibm_opt2_3.qasm',
 './qasm_output\\qft_transpiled_ibm_opt2_4.qasm',
 './qasm_output\\qft_transpiled_ibm_opt2_5.qasm']

In [63]:
from zipfile import ZipFile
def saveZippedFiles(paths:list, filename: str = "benchmarks"):
    with ZipFile(filename + '.zip', 'w') as zipobj:
        for path in paths:
            zipobj.write(path)
    zipobj.close()

In [64]:
saveZippedFiles(file_paths)